In [1]:
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')

import jax
import jax.numpy as jnp

from optax import losses

from molnet import loss
from molnet.data import input_pipeline_online, input_pipeline_seg
from configs.tests import attention_test
from configs import root_dirs

2025-01-17 13:30:42.255819: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-17 13:30:42.257167: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-17 13:30:42.285388: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-17 13:30:42.285919: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-17 13:30:44.978957: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
config = attention_test.get_config()
config.root_dir = root_dirs.get_root_dir("afms_rebias")

ds = input_pipeline_seg.get_datasets(config)['train']
batch = next(ds)

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 18085879115725111131
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 10565946296335614096
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing 

In [39]:
def cross_entropy_loss(logits, labels):
    """
    Computes the softmax cross-entropy loss for multi-class semantic segmentation in 3D.
    
    Args:
        logits: [D, H, W, num_classes] - Raw model outputs (logits).
        labels: [D, H, W] - Integer class labels for each voxel.
        
    Returns:
        Mean cross-entropy loss.
    """
    num_classes = logits.shape[-1]
    logits = logits.reshape(-1, num_classes)  # Flatten to [N, num_classes]
    labels = labels.flatten()                 # Flatten to [N]

    # Apply log-softmax for numerical stability
    log_probs = jax.nn.log_softmax(logits, axis=-1)

    # Gather log probabilities of the correct class using advanced indexing
    one_hot_labels = jax.nn.one_hot(labels, num_classes)
    loss = -jnp.sum(one_hot_labels * log_probs, axis=-1)

    return loss.mean()

In [68]:
rng = jax.random.PRNGKey(0)
y = batch['atom_map']
print(f"y.shape: {y.shape}")

y.shape: (16, 128, 128, 10)


In [88]:
rng, subkey = jax.random.split(rng)

logits = jax.random.normal(rng, y.shape + (5,))
labels = y

loss = cross_entropy_loss(logits, labels)
print("Loss:", loss)

Loss: 0.25647557
